# Giới thiệu về học sâu

## Pytorch Basic

In [42]:
import torch

In [43]:
torch.cuda.is_available()

True

sử dụng gpu và cuda

In [44]:
import torch
import numpy as np
import pandas as pd
df=pd.read_csv("./Iris.csv")
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [45]:
df.shape

(150, 6)

In [46]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()

X = df.drop(["Species"], axis = 1).values
y = le.fit_transform(df["Species"].values) 

#Chia du lieu voi test size = 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train).reshape(-1,1)
y_test = torch.LongTensor(y_test).reshape(-1,1)


In [47]:
len(y_train)

120

In [48]:
labels, counts = y_train.unique(return_counts=True)
print(labels, counts)

tensor([0, 1, 2]) tensor([40, 41, 39])


# Tính đạo hàm bằng pytorch
# Cho y = 2x^4 + x^3 + 3x^2 + 5x 1
# Tính y'

In [49]:
import torch

In [50]:
x = torch.tensor(2.0,requires_grad=True)
print(x)
print(x.grad)

tensor(2., requires_grad=True)
None


In [51]:
# Định nghĩa hàm 

y = 2*x**4 + x**3 + 3*x**2 + 5*x + 1
print(y)

tensor(63., grad_fn=<AddBackward0>)


In [52]:
y.grad_fn

In [53]:
#Thực hiện lan truyền ngược và tính toán các gradient
y.backward()

In [54]:
#8x^3 + 3x^2 + 6x + 5
x.grad 

tensor(93.)

In [55]:
#BTVN1

#Cho y= 5x^5 + 6x^3 - 3x + 1
import torch
x = torch.tensor(1.0, requires_grad=True)
y = 5*x**5 + 6*x**3 - 3*x + 1
print(y)
y.backward()
#Cho biết độ dốc của đa thức trên ở điểm nào 
x.grad

tensor(9., grad_fn=<AddBackward0>)


tensor(40.)

In [56]:
#BTVN2
#Tạo một tensor ban đầu có giá trị là 2
#Định nghĩa hàm số và tính gradient
#y = x^3 + 2x^2 + 5x + 1
#Hãy tính dy/dx tại giá trị của x
#Dùng pp Gradient Descent với Learning rate alpha = 0.1 để cập nhật giá trị x trong 10 vòng lặp
import torch

x = torch.tensor(2.0, requires_grad=True)
alpha = 0.1

for i in range(10):
    y = x**3 + 2*x**2 + 5*x + 1
    y.backward()
    x = x - alpha * x.grad
    x = x.detach().requires_grad_()
    
    print(x.item())

-0.5
-0.875
-1.2546875476837158
-1.7250847816467285
-2.427826166152954
-3.7249975204467773
-6.897680282592773
-18.912006378173828
-119.14640808105469
-4330.74755859375


In [57]:
#BTVN3
#Tạo một tập dữ liệu giả lặp với x là số giờ học(ngẫu nhiên từ 1-10) và y là số điểm được tính với noise là môt giá trị ngẫu nhiên nhỏ

#1.khởi tọa tham số w và b ngẫu nhiên với requires_grad = True
#2.Tính MSE
#3.Tính Gradient
#4.Cập nhật tham số w và b bằng Gradient Descent với Learning rate alpha = 0.01
#5.lặp lại quá trình trên trong 100 vòng lặp để xem sự hội tụ của mô hình 
import torch
torch.manual_seed(0)
x = torch.randint(1, 11, (20,), dtype=torch.float32) 
noise = torch.randn(20) * 0.5
y = 3 * x + 5 + noise
w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
alpha = 0.01

for i in range(100):
    y_pred = w * x + b
    mse = ((y_pred - y) ** 2).mean()
    mse.backward()
    with torch.no_grad():
        w -= alpha * w.grad
        b -= alpha * b.grad

    w.grad.zero_()
    b.grad.zero_()

    if (i + 1) % 10 == 0:
        print(f"Vòng {i+1}: loss={mse.item():.4f}, w={w.item():.4f}, b={b.item():.4f}")



Vòng 10: loss=4.9699, w=3.6745, b=-0.0422
Vòng 20: loss=4.6915, w=3.6520, b=0.1228
Vòng 30: loss=4.4292, w=3.6302, b=0.2829
Vòng 40: loss=4.1821, w=3.6091, b=0.4383
Vòng 50: loss=3.9493, w=3.5885, b=0.5891
Vòng 60: loss=3.7299, w=3.5686, b=0.7355
Vòng 70: loss=3.5233, w=3.5493, b=0.8776
Vòng 80: loss=3.3287, w=3.5305, b=1.0156
Vòng 90: loss=3.1453, w=3.5123, b=1.1494
Vòng 100: loss=2.9726, w=3.4946, b=1.2794


# Python with Tensor

In [58]:
import torch
import numpy as np

In [59]:
torch.__version__

'2.5.1+cu121'

In [60]:
# chuyển đổi mảng numpy sang tensor trong pytorch
arr = np.array([1, 2, 3, 4, 5])

print(arr)
print(arr.dtype)
print(type(arr))

[1 2 3 4 5]
int64
<class 'numpy.ndarray'>


In [61]:
x = torch.from_numpy(arr)
print(x)

tensor([1, 2, 3, 4, 5])


In [62]:
print(x.dtype)

torch.int64


In [63]:
print(x.type())

torch.LongTensor


In [64]:
arr2 = np.arange(0.,12.).reshape(4,3)
arr2

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.]])

In [65]:
x2 = torch.from_numpy(arr2)
print(x2)
print(x2.type())

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]], dtype=torch.float64)
torch.DoubleTensor


In [66]:
x = torch.tensor(arr)
print(x)

tensor([1, 2, 3, 4, 5])


In [67]:
arr[0] = 99
print(x)

tensor([1, 2, 3, 4, 5])


In [68]:
arr = np.arange(0,5)
x = torch.tensor(arr)
print(x)

tensor([0, 1, 2, 3, 4])


In [69]:
arr[0] = 99
print(x)

tensor([0, 1, 2, 3, 4])


BTVN 4: Giải thích 2 trường hợp trên
- Trường hợp 1:
Hàm torch.from_numpy() tạo tensor dùng chung vùng nhớ với NumPy array. Vì vậy, khi giá trị của NumPy array thay đổi thì tensor cũng thay đổi theo, và ngược lại. Cách này giúp tiết kiệm bộ nhớ và tăng tốc độ nhưng cần cẩn thận vì dữ liệu có thể bị thay đổi ngoài ý muốn.

- Trường hợp 2:
Hàm torch.tensor() tạo tensor bằng cách sao chép dữ liệu từ NumPy array sang vùng nhớ mới. Do đó, tensor và NumPy array độc lập với nhau, việc thay đổi NumPy array không ảnh hưởng đến tensor. Cách này an toàn hơn nhưng tốn bộ nhớ hơn.

# BTVN 5: 
Tạo tensor với
- emty
- zeros
- ones
- random
- reshape với view 

In [70]:
import torch
# emty
a = torch.empty(2, 3)
print("empty:\n", a)
# zeros
b = torch.zeros(2, 3)
print("zeros:\n", b)
# ones
c = torch.ones(2, 3)
print("ones:\n", c)
# random
d = torch.rand(2, 3)
print("random:\n", d)
# reshape với view
e = torch.arange(0, 12).view(4, 3)
print("reshape voi view:\n", e)

empty:
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
zeros:
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
ones:
 tensor([[1., 1., 1.],
        [1., 1., 1.]])
random:
 tensor([[0.7262, 0.7011, 0.2038],
        [0.6511, 0.7745, 0.4369]])
reshape voi view:
 tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])
